In [1]:
import pandas as pd
import konlpy
from konlpy.tag import Kkma

# 딕셔너리 만들기

{'title':'content', 'title':'content'...}

In [2]:
def make_dict(path):
    "input .csv path. Return (title, content) dictionary."
    df = pd.read_csv(path) # pandas를 이용해 path 안의 .csv 파일을 읽는다.
    titles = df['title'].values.tolist() # dataframe을 이용해 csv 파일의 title만 리스트로 변환한다.
    
    dict = {}

    for i in range(len(titles)):
        dict[df.loc[i]['title']] = df.loc[i]['content'] # dataframe의 인덱스를 이용해서 title:content dict를 만든다.
    return dict

In [3]:
care_dict = make_dict('care.csv')

# 특정 키워드 게시물의 긍부정 점수 책정

In [4]:
positive = ['증진', '허가', '합법', '추가', '확대', '지급'] # 추가 필요

In [5]:
negative = ['금지', '규제', '철폐', '강화']

In [6]:
tag_kkma = Kkma()

In [7]:
def direction(dict, str):
    "Input (dict, str)."
    direct = {} # 비어있는 dict를 만든다.
    direct[str] = 0 # 입력받은 str로 dict 생성
    for title in dict: # dict의 title을 하나씩 대입
        if str in title: # 입력받은 str이 title 안에 있을 경우
            content_noun = tag_kkma.nouns(dict[title]) # title의 상대 content를 명사로 쪼개 content_noun에 리스트로 선언
            for s in range(len(content_noun)-1):
                bigram = []
                bigram = [content_noun[s], content_noun[s+1]]
                test = {}
                test['score'] = 0
                for wd in bigram:
                    if wd in positive:
                        test['score'] += 1
                        print('P')
                    elif wd in negative:
                        test['score'] -= 2
                        print('n')
                print(bigram)
                if test['score'] == 1: # 1p
                    direct[str] += 0.5
                elif test['score'] == -2: # 1n
                    direct[str] -= 0.5
                elif test['score'] == 2: # 2p
                    direct[str] += 0
                elif test['score'] == -4: # 2n
                    direct[str] += 2
                elif test['score'] == -1: # 1n1p
                    direct[str] -= 1
                else: #nothing
                    pass
    return direct

In [10]:
test_dict = {'테스트를 위한 딕셔너리':'첫번째 테스트입니다. 규제를 확대한다는 문장은 -1점이 돼죠', '재미로 한번 더 돌려볼까요?' : 'A학점은 15%밖에 못 받아간다는 규제를 철폐하라! 라고 적었다면 1점이 되겠죠.'}

In [11]:
print(test_dict)

{'테스트를 위한 딕셔너리': '첫번째 테스트입니다. 규제를 확대한다는 문장은 -1점이 돼죠', '재미로 한번 더 돌려볼까요?': 'A학점은 15%밖에 못 받아간다는 규제를 철폐하라! 라고 적었다면 1점이 되겠죠.'}


In [12]:
direction(test_dict,'테스트')

['번째', '테스트']
n
['테스트', '규제']
n
P
['규제', '확대']
P
['확대', '문장']
['문장', '-1']
['-1', '-1점']
['-1점', '점']


{'테스트': -1.0}

In [13]:
direction(test_dict,'재미')

['학점', '15']
n
['15', '규제']
n
n
['규제', '철폐']
n
['철폐', '1']
['1', '1점']
['1점', '점']


{'재미': 1.0}